In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import IsolationForest

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy, MAPOCAM2
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier, LinearRule, GeneralClassifier_Shap
from cfmining.action_set import ActionSet

from results import save_result

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [13]:
data_dir = "data/"
data_name = 'german'
data_file = os.path.join(data_dir, '%s_processed.csv' % data_name)
## load and process data
german_df = pd.read_csv(data_file).reset_index(drop=True)
german_df = (german_df
             .assign(isMale=lambda df: (df['Gender']=='Male').astype(int))
             .drop(['PurposeOfLoan', 'Gender', 'OtherLoansAtStore'], axis=1)
            )

y = german_df['GoodCustomer']
# tranform y to [0, 1]
y = (y == 1).astype(int)
X = german_df.drop('GoodCustomer', axis=1)
X = X[['ForeignWorker', 'Single', 'Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'isMale']]

In [14]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

In [52]:
outlier_detection = IsolationForest(contamination=0.1)
outlier_detection.fit(Xtr.values);

## Setting ActionSet base parameters

In [16]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['Age', 'JobClassIsSkilled', 'OwnsHouse', 'isMale', 'JobClassIsSkilled']:
        feat.mutable = False
        feat.step_direction = 1
    if feat.name in ['Single', 'ForeignWorker', 'RentsHouse']:
        feat.mutable = False
        feat.step_direction = -1
    if feat.name in ['LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'NumberOfOtherLoansAtBank', 'MissedPayments',
                     'CriticalAccountOrLoansElsewhere', 'OtherLoansAtBank', 'Unemployed', 'YearsAtCurrentJob_lt_1']:
        feat.mutable = True
        feat.step_direction = -1
    if feat.name in ['YearsAtCurrentHome', 'NumberOfLiableIndividuals', 'HasTelephone', 'CheckingAccountBalance_geq_0',
                     'CheckingAccountBalance_geq_200', 'SavingsAccountBalance_geq_100', 'SavingsAccountBalance_geq_500',
                     'NoCurrentLoan', 'HasCoapplicant', 'HasGuarantor', 'YearsAtCurrentJob_geq_4']:
        feat.mutable = True
        feat.step_direction = 1
    
    feat.flip_direction = 1
    feat.update_grid()

## Logistic regression model

In [58]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [59]:
threshold=0.5
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [60]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=threshold)
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_

In [20]:
clf_lgr_shap = GeneralClassifier_Shap(clf_lgr, X=Xtr, y=ytr, threshold=threshold)

ExactExplainer explainer: 901it [01:28,  9.80it/s]                         


In [22]:
imp = list(zip(Xtr.columns, clf_lgr_shap.importances, clf_lgr_shap.shap_max))
imp.sort(key=lambda x: x[1], reverse=True)
imp

[('LoanDuration', 0.04484870130721758, 0.08340935229296217),
 ('LoanAmount', 0.030163973351294584, 0.04583432136365088),
 ('LoanRateAsPercentOfIncome', 0.030085474401012984, 0.06782382308648012),
 ('Age', 0.02964235821026434, 0.12828806922361097),
 ('Single', 0.028745031809877856, 0.029369125511563156),
 ('isMale', 0.015604658144219711, 0.010974517425070171),
 ('ForeignWorker', 0.006664880976964589, 0.14399461782002923)]

In [23]:
clf_lgr_shap = GeneralClassifier_Shap(clf_lgr, outlier_detection=outlier_detection, X=Xtr, y=ytr, threshold=threshold)

ExactExplainer explainer: 901it [00:46, 14.63it/s]                         


In [24]:
imp = list(zip(Xtr.columns, clf_lgr_shap.importances, clf_lgr_shap.shap_max))
imp.sort(key=lambda x: x[1], reverse=True)
imp

[('LoanDuration', 0.04484870130721758, 0.08340935229296217),
 ('LoanAmount', 0.030163973351294584, 0.04583432136365088),
 ('LoanRateAsPercentOfIncome', 0.030085474401012984, 0.06782382308648012),
 ('Age', 0.02964235821026434, 0.12828806922361097),
 ('Single', 0.028745031809877856, 0.029369125511563156),
 ('isMale', 0.015604658144219711, 0.010974517425070171),
 ('ForeignWorker', 0.006664880976964589, -0.0012386953924630462)]

## Preparing counterfactual parameters

In [26]:
action_set = copy.deepcopy(action_set_base)
action_set['Age'].mutable = False
action_set['Age'].update_grid()

action_set['LoanDuration'].step_type ="absolute"
action_set['LoanDuration'].step_size = 6
action_set['LoanDuration'].update_grid()

action_set['LoanAmount'].step_size = 0.1
action_set['LoanAmount'].update_grid()

print('ActionSet stats')
print('Number of actionable features:', sum([action.actionable for action in action_set]))
print('Mean number of actions per feature:', np.nanmean([len(action._grid) if action.actionable else np.nan for action in action_set]))
print('Max number of actions per feature:', np.nanmax([len(action._grid) if action.actionable else np.nan for action in action_set]))
log_combinations = int(np.prod([len(action._grid) for action in action_set if action_set.actionable]))
print('Number of combinations:', log_combinations)

ActionSet stats
Number of actionable features: 0
Mean number of actions per feature: nan
Max number of actions per feature: nan
Number of combinations: 172480


Mean of empty slice
All-NaN axis encountered


In [27]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [ ]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, threshold)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = np.array([int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)])
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if not any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [18]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam,
                      max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

  0%|          | 0/47 [00:00<?, ?it/s]

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
  2%|▏         | 1/47 [00:00<00:17,  2.66it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does n

## MAPOCAM2

In [67]:
mapocam2_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM2(action_set, individual, clf_lgr_shap,
                      max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam2_lgr_results[i] = {}
    mapocam2_lgr_results[i]['enum'] = mapocam
    mapocam2_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam2_lgr_results[i]['time'] = mapocam_time
    mapocam2_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

  0%|          | 0/45 [00:00<?, ?it/s]X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


max prob: [0.84909722]
max prob: [0.82077022]
max prob: [0.79256889]
max prob: [0.84727476]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


max prob: [0.82707009]
max prob: [0.80676361]
max prob: [0.80185625]
max prob: [0.81886206]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


...

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names


max prob: [0.84905329]
max prob: [0.82119128]
max prob: [0.79330558]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
 91%|█████████ | 41/45 [00:08<00:00,  5.82it/s]

max prob: [0.76556412]
max prob: [0.73813144]
max prob: [0.71116503]


X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
100%|██████████| 45/45 [00:08<00:00,  5.02it/s]


## Performance comparison for single objective


In [ ]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

best_costs = np.min([lp_lgr_costs, greedy_lgr_costs, mapocam_lgr_costs], axis=0)

In [ ]:
lgr_dict = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_lgr_time)),
                'greedy': float(np.mean(greedy_lgr_time)),
                'mi-logistic': float(np.mean(lp_lgr_time)),
            },
            'Best solution rate':
            {
                'MAPOCAM':float(np.mean(mapocam_lgr_costs==best_costs)),
                'greedy': float(np.mean(greedy_lgr_costs==best_costs)),
                'mi-logistic': float(np.mean(lp_lgr_costs==best_costs)),
            },
            'Feasible rate':1-float(np.mean(np.isinf(mapocam_lgr_costs))),
            'Number of combinations':log_combinations
    
}
save_result(lgr_dict, 'german', 'logistic', 'MPC cost')

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=True, clean_suboptimal=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_pc[i] = {}
    bf_results_pc[i]['enum'] = bf
    bf_results_pc[i]['time'] = bf_time
    bf_results_pc[i]['num'] = len(bf.solutions)

In [ ]:
brute_sols_pc = np.array([bf_results_pc[i]['num'] for i in denied_individuals])
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])
best_sols_pc = brute_sols_pc

brute_time_pc = np.array([bf_results_pc[i]['time'] for i in denied_individuals])
mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

In [ ]:
lgr_dict_pc = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_pc)),
                'brute-force': float(np.mean(brute_time_pc)),
                'po-mi-logistic': float(np.mean(lp_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_pc==best_sols_pc)),
               'brute-force': float(np.mean(brute_sols_pc==best_sols_pc)),
               'po-mi-logistic': float(np.mean(lp_sols_pc==best_sols_pc)),
           }
    
}
save_result(lgr_dict_pc, 'german', 'logistic', 'MPC vs. #changes')

## Features

In [ ]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam,
                      max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_feat[i] = {}
    bf_results_feat[i]['enum'] = bf
    bf_results_feat[i]['time'] = bf_time
    bf_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_sols_feat = np.array([bf_results_feat[i]['num'] for i in denied_individuals])
mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])
best_sols_feat = brute_sols_feat

brute_time_feat = np.array([bf_results_feat[i]['time'] for i in denied_individuals])
mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

In [ ]:
lgr_dict_feat = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_feat)),
                'brute-force': float(np.mean(brute_time_feat)),
                'po-mi-logistic': float(np.mean(lp_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_feat==best_sols_feat)),
               'brute-force': float(np.mean(brute_sols_feat==best_sols_feat)),
               'po-mi-logistic': float(np.mean(lp_sols_feat==best_sols_feat)),
           }
    
}
save_result(lgr_dict_feat, 'german', 'logistic', 'feature')

## Random forest

In [39]:
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

In [40]:
clf_rt = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5, max_leaf_nodes=31,
                                         class_weight='balanced_subsample')
clf_rt.fit(Xtr, ytr);

## Preparing counterfactual parameters

In [41]:
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction
    
print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_tree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_tree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_tree]))
rt_combinations = int(np.prod([len(action._grid) for action in action_set_tree if action.actionable]))
print('Number of combinations:', rt_combinations)

ActionSet stats
Number of actionable features: 3
Mean number of actions per feature: 11.142857142857142
Max number of actions per feature: 33
Number of combinations: 2584


In [42]:
scores = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [44]:
percCalc = PercentileCalculator(action_set=action_set_tree)

In [45]:
mapocam_clf_rf = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold)
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)

In [47]:
clf_rf_shap = GeneralClassifier_Shap(clf_rt, X=Xtr, y=ytr, threshold=threshold)

ExactExplainer explainer: 901it [00:40, 14.98it/s]                         


In [48]:
imp = list(zip(Xtr.columns, clf_rf_shap.importances, clf_rf_shap.shap_max))
imp.sort(key=lambda x: x[1], reverse=True)
imp

[('LoanDuration', 0.08396093310336354, 0.2779718140399186),
 ('Age', 0.057508943987659815, 0.19671331484564517),
 ('LoanAmount', 0.03662699766295823, 0.1395516791573939),
 ('Single', 0.021008334734129636, 0.095156324096787),
 ('isMale', 0.012080998224669705, 0.08227360496431882),
 ('LoanRateAsPercentOfIncome', 0.01144972571009471, 0.12564793716935008),
 ('ForeignWorker', 0.006460803714000895, 0.2548199096230261)]

In [49]:
clf_rf_shap = GeneralClassifier_Shap(clf_rt, outlier_detection=outlier_detection, X=Xtr, y=ytr, threshold=threshold)

ExactExplainer explainer: 901it [00:13, 25.78it/s]                          


In [50]:
imp = list(zip(Xtr.columns, clf_rf_shap.importances, clf_rf_shap.shap_max))
imp.sort(key=lambda x: x[1], reverse=True)
imp

[('LoanDuration', 0.08396093310336354, 0.2779718140399186),
 ('Age', 0.057508943987659815, 0.19671331484564517),
 ('LoanAmount', 0.03662699766295823, 0.13678909653787466),
 ('Single', 0.021008334734129636, 0.06347488540425911),
 ('isMale', 0.012080998224669705, 0.08227360496431882),
 ('LoanRateAsPercentOfIncome', 0.01144972571009471, 0.12564793716935008),
 ('ForeignWorker', 0.006460803714000895, 5.947448460845439e-05)]

## Linear programming

In [ ]:
lp_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual,
                               clf_rt, threshold=threshold,
                               max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_rf_results[i] = {}
    lp_rf_results[i]['enum'] = lp
    lp_rf_results[i]['indiv'] = individual
    lp_rf_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_rf_results[i]['time'] = lp_time
    lp_rf_results[i]['cost'] = criteria.f(lp_rf_results[i]['solution']) if not any(np.isnan(lp_rf_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    greedy = Greedy(action_set_tree, individual, mapocam_clf_rf_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_rf_results[i] = {}
    greedy_rf_results[i]['solution'] = greedy.solution
    greedy_rf_results[i]['time'] = greedy_time
    greedy_rf_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [81]:
mapocam_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf, max_changes=float('inf'),
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results[i] = {}
    mapocam_rf_results[i]['enum'] = mapocam
    mapocam_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results[i]['time'] = mapocam_time
    mapocam_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 45/45 [00:00<00:00, 52.48it/s]


In [ ]:
mapocam_rf_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf_fast, max_changes=float('inf'),
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_plus[i] = {}
    mapocam_rf_results_plus[i]['enum'] = mapocam
    mapocam_rf_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results_plus[i]['time'] = mapocam_time
    mapocam_rf_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

## Mapocam 2

In [51]:
mapocam2_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM2(action_set, individual, clf_rf_shap,
                      max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam2_rf_results[i] = {}
    mapocam2_rf_results[i]['enum'] = mapocam
    mapocam2_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam2_rf_results[i]['time'] = mapocam_time
    mapocam2_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

  0%|          | 0/35 [00:00<?, ?it/s]X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  3%|▎         | 1/35 [00:00<00:09,  3.50it/s]

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassif

max prob: [1.29294823]
max prob: [0.94747064]
max prob: [0.47874238]
max prob: [0.60106181]
max prob: [0.5155523]
max prob: [0.44237677]


X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
100%|██████████| 35/35 [00:01<00:00, 31.78it/s]


In [83]:
np.sum([v["time"] for v in mapocam_rf_results.values() if v["solution"] is not None]), np.sum([v["time"] for v in mapocam2_rf_results.values() if v["solution"] is not None])

(0.6279486449911929, 1.5361950600035925)

In [ ]:
lp_rf_costs = np.array([lp_rf_results[i]['cost'] for i in denied_individuals])
greedy_rf_costs = np.array([greedy_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs = np.array([mapocam_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs_plus = np.array([mapocam_rf_results_plus[i]['cost'] for i in denied_individuals])
best_costs_rf = np.min([lp_rf_costs, greedy_rf_costs, mapocam_rf_costs, mapocam_rf_costs_plus], axis=0)

lp_rf_time = np.array([lp_rf_results[i]['time'] for i in denied_individuals])
greedy_rf_time = np.array([greedy_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time = np.array([mapocam_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time_plus = np.array([mapocam_rf_results_plus[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time)),
                'greedy': float(np.mean(greedy_rf_time)),
                'mi-trees': float(np.mean(lp_rf_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_costs_plus==best_costs_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_costs==best_costs_rf)),
               'greedy': float(np.mean(greedy_rf_costs==best_costs_rf)),
               'mi-trees': float(np.mean(lp_rf_costs==best_costs_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isinf(mapocam_rf_costs_plus))),
             'Number of combinations': rt_combinations
    
}
save_result(tree_dict, 'german', 'tree', 'MPC cost')

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc[i] = {}
    mapocam_rf_results_pc[i]['enum'] = mapocam
    mapocam_rf_results_pc[i]['time'] = mapocam_time
    mapocam_rf_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_rf_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc_plus[i] = {}
    mapocam_rf_results_pc_plus[i]['enum'] = mapocam
    mapocam_rf_results_pc_plus[i]['time'] = mapocam_time
    mapocam_rf_results_pc_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_pc[i] = {}
    lp_rf_results_pc[i]['enum'] = lp
    lp_rf_results_pc[i]['time'] = lp_time
    lp_rf_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_pc[i] = {}
    bf_rf_results_pc[i]['enum'] = bf
    bf_rf_results_pc[i]['time'] = bf_time
    bf_rf_results_pc[i]['num'] = len(bf.solutions)

In [ ]:
lp_rf_sols_pc = np.array([lp_rf_results_pc[i]['num'] for i in denied_individuals])
brute_rf_sols_pc = np.array([bf_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc = np.array([mapocam_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['num'] for i in denied_individuals])
best_rf_sols_pc = brute_rf_sols_pc

lp_rf_time_pc = np.array([lp_rf_results_pc[i]['time'] for i in denied_individuals])
brute_rf_time_pc = np.array([bf_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc = np.array([mapocam_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_pc)),
                'brute-force': float(np.mean(brute_rf_time_pc)),
                'po-mi-trees': float(np.mean(lp_rf_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_pc_plus==best_rf_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_pc==best_rf_sols_pc)),
               'brute-force': float(np.mean(brute_rf_sols_pc==best_rf_sols_pc)),
               'po-mi-trees': float(np.mean(lp_rf_sols_pc==best_rf_sols_pc)),
           }
    
}
save_result(tree_dict_pc, 'german', 'tree', 'MPC vs. #changes')

## Features

In [ ]:
mapocam_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat[i] = {}
    mapocam_rf_results_feat[i]['enum'] = mapocam
    mapocam_rf_results_feat[i]['time'] = mapocam_time
    mapocam_rf_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_rf_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat_plus[i] = {}
    mapocam_rf_results_feat_plus[i]['enum'] = mapocam
    mapocam_rf_results_feat_plus[i]['time'] = mapocam_time
    mapocam_rf_results_feat_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_feat[i] = {}
    lp_rf_results_feat[i]['enum'] = lp
    lp_rf_results_feat[i]['time'] = lp_time
    lp_rf_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_feat[i] = {}
    bf_rf_results_feat[i]['enum'] = bf
    bf_rf_results_feat[i]['time'] = bf_time
    bf_rf_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_rf_sols_feat = np.array([bf_rf_results_feat[i]['num'] for i in denied_individuals])
lp_rf_sols_feat = np.array([lp_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat = np.array([mapocam_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['num'] for i in denied_individuals])
best_rf_sols_feat = brute_rf_sols_feat

brute_rf_time_feat = np.array([bf_rf_results_feat[i]['time'] for i in denied_individuals])
lp_rf_time_feat = np.array([lp_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat = np.array([mapocam_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_feat)),
                'brute-force': float(np.mean(brute_rf_time_feat)),
                'po-mi-trees': float(np.mean(lp_rf_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_feat_plus==best_rf_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_feat==best_rf_sols_feat)),
               'brute-force': float(np.mean(brute_rf_sols_feat==best_rf_sols_feat)),
               'po-mi-trees': float(np.mean(lp_rf_sols_feat==best_rf_sols_feat)),
           }
    
}
save_result(tree_dict_feat, 'german', 'tree', 'feature')

## Monotone trees

In [ ]:
from sklearn import tree, ensemble
import lightgbm
from cfmining.predictors import MonotoneTree

In [ ]:
monotone_constraints = [action.step_direction for action in action_set_base]
clf_mt = lightgbm.LGBMClassifier(n_estimators=10, max_depth=10, num_leaves=63,
                                 monotone_constraints=monotone_constraints, class_weight='balanced')
clf_mt.fit(Xtr, ytr)

## Preparing counterfactual parameters

In [ ]:
action_set_mtree = copy.deepcopy(action_set_base)
action_set_mtree.embed_forest(clf_mt)
for action in action_set_mtree:
    action.flip_direction = action.step_direction

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_mtree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_mtree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_mtree]))
mt_combinations = int(np.prod([len(action._grid) for action in action_set_mtree if action.actionable]))
print('Number of combinations:', mt_combinations)

In [ ]:
scores = pd.Series(clf_mt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [ ]:
percCalc = PercentileCalculator(action_set=action_set_mtree)

In [ ]:
mapocam_clf_mt = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold)
mapocam_clf_mt_fast = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)
mapocam_clf_mt_mon = MonotoneTree(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)

## Linear programming

In [ ]:
lp_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_mt_results[i] = {}
    lp_mt_results[i]['enum'] = lp
    lp_mt_results[i]['indiv'] = individual
    lp_mt_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_mt_results[i]['time'] = lp_time
    lp_mt_results[i]['cost'] = criteria.f(lp_mt_results[i]['solution']) if not any(np.isnan(lp_mt_results[i]['solution'])) else float('inf')

## Greedy approach

In [ ]:
greedy_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    greedy = Greedy(action_set_mtree, individual, mapocam_clf_mt_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_mt_results[i] = {}
    greedy_mt_results[i]['solution'] = greedy.solution
    greedy_mt_results[i]['time'] = greedy_time
    greedy_mt_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

## MAPOCAM

In [ ]:
mapocam_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, 
                      mapocam_clf_mt, max_changes=float('inf'), 
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results[i] = {}
    mapocam_mt_results[i]['enum'] = mapocam
    mapocam_mt_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results[i]['time'] = mapocam_time
    mapocam_mt_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_mt_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, 
                      mapocam_clf_mt_fast, max_changes=float('inf'),
                      compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_plus[i] = {}
    mapocam_mt_results_plus[i]['enum'] = mapocam
    mapocam_mt_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_plus[i]['time'] = mapocam_time
    mapocam_mt_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
mapocam_mt_results_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_mon[i] = {}
    mapocam_mt_results_mon[i]['enum'] = mapocam
    mapocam_mt_results_mon[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_mon[i]['time'] = mapocam_time
    mapocam_mt_results_mon[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

In [ ]:
lp_mt_costs = np.array([lp_mt_results[i]['cost'] for i in denied_individuals])
greedy_mt_costs = np.array([greedy_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs = np.array([mapocam_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_plus = np.array([mapocam_mt_results_plus[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_mon = np.array([mapocam_mt_results_mon[i]['cost'] for i in denied_individuals])
best_costs_mt_rf = np.min([lp_mt_costs, greedy_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus, mapocam_mt_costs_mon], axis=0)

lp_mt_time = np.array([lp_mt_results[i]['time'] for i in denied_individuals])
greedy_mt_time = np.array([greedy_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time = np.array([mapocam_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time_plus = np.array([mapocam_mt_results_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_mon = np.array([mapocam_mt_results_mon[i]['time'] for i in denied_individuals])

In [ ]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time)),
                'greedy': float(np.mean(greedy_mt_time)),
                'mi-trees': float(np.mean(lp_mt_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_costs_mon==best_costs_mt_rf)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_costs_plus==best_costs_mt_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_costs==best_costs_mt_rf)),
               'greedy': float(np.mean(greedy_mt_costs==best_costs_mt_rf)),
               'mi-trees': float(np.mean(lp_mt_costs==best_costs_mt_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isinf(mapocam_mt_costs_mon))),
             'Number of combinations': mt_combinations
    
}
save_result(tree_dict, 'german', 'monotone-tree', 'MPC cost')

## Comparing Pareto enumeration

## Percentile vs changes

In [ ]:
mapocam_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, 
                      mapocam_clf_mt, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc[i] = {}
    mapocam_mt_results_pc[i]['enum'] = mapocam
    mapocam_mt_results_pc[i]['time'] = mapocam_time
    mapocam_mt_results_pc[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_plus[i] = {}
    mapocam_mt_results_pc_plus[i]['enum'] = mapocam
    mapocam_mt_results_pc_plus[i]['time'] = mapocam_time
    mapocam_mt_results_pc_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_pc_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_mon, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_mon[i] = {}
    mapocam_mt_results_pc_mon[i]['enum'] = mapocam
    mapocam_mt_results_pc_mon[i]['time'] = mapocam_time
    mapocam_mt_results_pc_mon[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_pc[i] = {}
    lp_mt_results_pc[i]['enum'] = lp
    lp_mt_results_pc[i]['time'] = lp_time
    lp_mt_results_pc[i]['num'] = len(lp.solutions)

In [ ]:
bf_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_pc[i] = {}
    bf_mt_results_pc[i]['enum'] = bf
    bf_mt_results_pc[i]['time'] = bf_time
    bf_mt_results_pc[i]['num'] = len(bf.solutions)

In [ ]:
brute_mt_sols_pc = np.array([bf_mt_results_pc[i]['num'] for i in denied_individuals])
lp_mt_sols_pc = np.array([lp_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc = np.array([mapocam_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['num'] for i in denied_individuals])
best_mt_sols_pc = brute_mt_sols_pc

brute_mt_time_pc = np.array([bf_mt_results_pc[i]['time'] for i in denied_individuals])
lp_mt_time_pc = np.array([lp_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc = np.array([mapocam_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['time'] for i in denied_individuals])

In [ ]:
mon_tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_pc_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_pc)),
                'brute-force': float(np.mean(brute_mt_time_pc)),
                'po-mi-trees': float(np.mean(lp_mt_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc_mon)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_pc_plus==best_mt_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_pc==best_mt_sols_pc)),
               'brute-force': float(np.mean(brute_mt_sols_pc==best_mt_sols_pc)),
               'po-mi-trees': float(np.mean(lp_mt_sols_pc==best_mt_sols_pc)),
           }
    
}
save_result(mon_tree_dict_pc, 'german', 'monotone-tree', 'MPC vs. #changes')

## Features

In [ ]:
mapocam_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat[i] = {}
    mapocam_mt_results_feat[i]['enum'] = mapocam
    mapocam_mt_results_feat[i]['time'] = mapocam_time
    mapocam_mt_results_feat[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_plus[i] = {}
    mapocam_mt_results_feat_plus[i]['enum'] = mapocam
    mapocam_mt_results_feat_plus[i]['time'] = mapocam_time
    mapocam_mt_results_feat_plus[i]['num'] = len(mapocam.solutions)

In [ ]:
mapocam_mt_results_feat_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_mon[i] = {}
    mapocam_mt_results_feat_mon[i]['enum'] = mapocam
    mapocam_mt_results_feat_mon[i]['time'] = mapocam_time
    mapocam_mt_results_feat_mon[i]['num'] = len(mapocam.solutions)

In [ ]:
lp_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm', threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_feat[i] = {}
    lp_mt_results_feat[i]['enum'] = lp
    lp_mt_results_feat[i]['time'] = lp_time
    lp_mt_results_feat[i]['num'] = len(lp.solutions)

In [ ]:
bf_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_feat[i] = {}
    bf_mt_results_feat[i]['enum'] = bf
    bf_mt_results_feat[i]['time'] = bf_time
    bf_mt_results_feat[i]['num'] = len(bf.solutions)

In [ ]:
brute_mt_sols_feat = np.array([bf_mt_results_feat[i]['num'] for i in denied_individuals])
lp_mt_sols_feat = np.array([lp_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat = np.array([mapocam_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['num'] for i in denied_individuals])
best_mt_sols_feat = brute_mt_sols_feat

brute_mt_time_feat = np.array([bf_mt_results_feat[i]['time'] for i in denied_individuals])
lp_mt_time_feat = np.array([lp_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat = np.array([mapocam_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['time'] for i in denied_individuals])

In [ ]:
print('Feasible rate:', np.mean(best_mt_sols_feat!=0))

In [ ]:
mon_tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_feat_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_feat)),
                'brute-force': float(np.mean(brute_mt_time_feat)),
                'po-mi-trees': float(np.mean(lp_mt_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_feat_mon==best_mt_sols_feat)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_feat_plus==best_mt_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_feat==best_mt_sols_feat)),
               'brute-force': float(np.mean(brute_mt_sols_feat==best_mt_sols_feat)),
               'po-mi-trees': float(np.mean(lp_mt_sols_feat==best_mt_sols_feat)),
           }
    
}
save_result(mon_tree_dict_feat, 'german', 'monotone-tree', 'feature')